<a href="https://colab.research.google.com/github/williap2745/Senior-Project/blob/Pipeline/school_work_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [ ]:
import pandas as pd
import numpy as np

# Define subjects
subjects = ["Math", "Biology", "Chemistry", "History",'Criminal Justice', "Cybersecurity", "English", "Computer Science", "Engineering"]

#subject difficulty wieghts
subject_difficulty_weights = {
    "Math": 9,
    "Biology": 7,
    "Chemistry": 8,
    "History": 5,
    "Criminal Justice": 4,
    "Cybersecurity": 6,
    "English": 3,
    "Computer Science": 8,
    "Engineering": 9
}

subject_difficulty = {
    "Math": 1.2, "Biology": 1.1, "Chemistry": 1.2, "History": 0.9,
    "Criminal Justice": 1.0, "Cybersecurity": 1.3, "English": 0.8,
    "Computer Science": 1.4, "Engineering": 1.5
}

# Number of students and study sessions
#num_students = 1000
num_sessions = 150000

# Generate random data
data = []
for _ in range(num_sessions):
    #student_id = np.random.randint(1, num_students + 1)
    subject = np.random.choice(subjects)
    skill_level = np.random.randint(1, 11)  # Skill level (1-10)

    #Now assigning task difficulty based on subject weights above
    base_difficulty = subject_difficulty_weights[subject]
    task_difficulty = np.clip(np.random.normal(loc=base_difficulty, scale=2), 1, 10).astype(int)

    # Calculate study time (lower skill & higher difficulty -> more time)
    base_time = np.random.uniform(0.5, 4.0)  # Base study time (30 min - 3 hours)
    study_time = base_time * (1 + (task_difficulty - skill_level) / 10)

    # Simulate score based on skill, study time, and randomness
    score = np.clip(skill_level * 10 + (study_time * 5 / subject_difficulty[subject]) + np.random.uniform(-10, 10), 0, 100)

    data.append([subject, skill_level, task_difficulty, round(study_time, 2), round(score, 2)])

# Convert to DataFrame
test_df = pd.DataFrame(data, columns=["Subject", "Skill_Level", "Task_Difficulty", "Study_Time", "Score"])

In [ ]:
test_df

,Subject,Skill_Level,Task_Difficulty,Study_Time,Score
0,History,5,2,1.46,63.96
1,English,5,3,2.44,66.14
2,Engineering,9,7,1.35,100.00
3,Biology,3,8,2.82,42.28
4,History,5,5,3.28,67.93
...,...,...,...,...,...
149995,Criminal Justice,6,4,3.04,84.90
149996,Computer Science,5,6,4.08,67.19
149997,Chemistry,9,10,0.56,99.86
149998,Computer Science,9,7,2.42,100.00


In [ ]:
# going to represent each subject with an integer so it can be used when I am training my model
# label_encoder = LabelEncoder()
# test_df["Subject_Encoded"] = label_encoder.fit_transform(test_df["Subject"])
# test_df

# going to try one-hot encoding

# I have now put this into one big pipeline so I have no need to do this here

# One hot-encoding is preventing the model from assigning weights to each subject to influence predictions
# encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
# encoded_subjects = encoder.fit_transform(test_df[["Subject"]])
# subject_df = pd.DataFrame(encoded_subjects, columns=encoder.get_feature_names_out(["Subject"]))

# test_df = test_df.drop(columns=["Subject"])
# test_df = pd.concat([test_df, subject_df], axis=1)
# test_df

,Skill_Level,Task_Difficulty,Study_Time,Score,Subject_Biology,Subject_Chemistry,Subject_Computer Science,Subject_Criminal Justice,Subject_Cybersecurity,Subject_Engineering,Subject_English,Subject_History,Subject_Math
0,10,10,0.89,100.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,5,8,2.32,68.28,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,5,3.47,63.77,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,8,0.94,47.76,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,9,5.37,78.47,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,9,5,1.31,87.53,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
149996,5,7,2.84,64.32,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
149997,3,8,2.49,51.24,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149998,1,8,4.03,25.06,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#scaling features to prevent the importance of features
# scaler = StandardScaler()
#numerical_features = ["Skill_Level", "Task_Difficulty", "Score"]
# test_df[["Skill_Level", "Task_Difficulty", "Score"]] = scaler.fit_transform(test_df[["Skill_Level", "Task_Difficulty", "Score"]])

In [ ]:
# Assign numeric difficulty instead of one-hot encoding
# test_df["Subject_Difficulty"] = test_df["Subject"].map(subject_difficulty)

# Drop original categorical column
# test_df.drop(columns=["Subject"], inplace=True)
# test_df

KeyError: 'Subject'

In [ ]:
test_df

,Skill_Level,Task_Difficulty,Study_Time,Score,Subject_Difficulty
0,0.869645,1.127987,3.13,1.253609,1.5
1,0.520963,-0.004843,1.47,0.127314,1.1
2,0.869645,-0.760064,1.31,0.634458,0.9
3,0.172281,-1.892894,1.63,0.300380,0.8
4,-1.571128,1.127987,4.74,-1.548129,1.5
...,...,...,...,...,...
149995,0.869645,1.505597,3.92,1.379228,1.2
149996,-1.222446,1.505597,3.61,-1.412787,1.4
149997,-0.176401,-1.137674,3.12,0.310881,0.8
149998,1.218327,1.505597,2.63,1.379228,1.4


In [ ]:
 # Train model with RandomForestRegressor
# from sklearn.ensemble import RandomForestRegressor

# X = test_df.drop(columns=['Study_Time'])
# y = test_df["Study_Time"]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = RandomForestRegressor(n_estimators=100, random_state= 42)
# model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [ ]:
#train data with xgboost
# import xgboost as xgb
# model2 = xgb.XGBRegressor(n_estimators=100, max_depth = 7, random_state=42)
# model2.fit(X_train, y_train) # trying to create a whole pipeline instead

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
# train_score = model.score(X_train, y_train)
# print(f"Training Accuracy/Score: {train_score}")

Training Accuracy/Score: 0.907261802692541


In [ ]:
# Predict on the test set
# y_pred = model.predict(X_test)

# # If it's regression, evaluate performance with R-squared or MSE
# from sklearn.metrics import mean_squared_error, r2_score


# #checking for accuracy
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"Mean Squared Error (MSE): {mse}")
# print(f"R-squared (R2): {r2}")



Mean Squared Error (MSE): 0.8634514759295907
R-squared (R2): 0.5801618213043541


In [ ]:
# # Predict on the test set with XGB Model
# y_pred = model2.predict(X_test)

# # If it's regression, evaluate performance with R-squared or MSE
# from sklearn.metrics import mean_squared_error, r2_score


# #checking for accuracy
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"Mean Squared Error (MSE): {mse}")
# print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 2.7398812434081012
R-squared (R2): 0.7020803203487562


In [ ]:
# #This data will be recieved from frontend when users input their information
# new_entry = pd.DataFrame([{
#     "Subject": "Math",  # Change subject as needed
#     "Skill_Level": 2.0,               # Adjust based on user skill
#     "Task_Difficulty": 9.0,           # Difficulty of the task
#     "Score": 93.0                     # Desired Grade (in hours)
#     }])

In [ ]:
# # encoded_subjects = encoder.transform(new_entry[['Subject']])
# # subject_df = pd.DataFrame(encoded_subjects, columns=encoder.get_feature_names_out(['Subject']))
# # new_entry = new_entry.drop(columns=['Subject'])
# # new_entry = pd.concat([new_entry, subject_df], axis=1)
# #features = ["Skill_Level", "Task_Difficulty", "Score", "Subject_Encoded"]
# #new_entry = new_entry[features]

# new_entry['Subject_Difficulty'] = new_entry['Subject'].map(subject_difficulty)
# new_entry[["Skill_Level", "Task_Difficulty", "Score"]] = scaler.fit_transform(new_entry[["Skill_Level", "Task_Difficulty", "Score"]])

# # Drop original categorical column
# new_entry.drop(columns=["Subject"], inplace=True)

In [ ]:
# new_entry

,Skill_Level,Task_Difficulty,Score,Subject_Difficulty
0,0.0,0.0,0.0,1.2


In [ ]:
predicted_hours = model.predict(new_entry)
print(f"Predicted Study Hours: {predicted_hours[0]}")

Predicted Study Hours: 1.7646999999999988


In [ ]:
#will save one-hot encoding pipeline in colab and upload it into my backend
#I will also use the same steps with my algorithm so it can be trained seperately

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

#defining model here for use
model = RandomForestRegressor(n_estimators=100, random_state= 42)
numerical_features = ['Skill_Level', 'Task_Difficulty', 'Score']


preprocessor = ColumnTransformer(transformers=[('num_scaler', StandardScaler(), numerical_features), ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False), ['Subject'])],
remainder='passthrough')

pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])

# Train the pipeline
X = test_df.drop(columns='Study_Time')  # Features
y = test_df["Study_Time"]  # Target

#splitting up data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=X["Subject"])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)
#pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_scaler',
                                                  StandardScaler(),
                                                  ['Skill_Level',
                                                   'Task_Difficulty',
                                                   'Score']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Subject'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [ ]:
# X

,Subject,Skill_Level,Task_Difficulty,Score
0,Math,2,10,32.68
1,Engineering,1,7,29.04
2,Engineering,10,8,100.00
3,Computer Science,8,10,90.95
4,Chemistry,5,10,55.76
...,...,...,...,...
149995,English,8,4,95.49
149996,English,3,1,41.56
149997,English,8,3,82.92
149998,Math,10,9,100.00


In [ ]:
# Predict on the test set with XGB Model
y_pred = pipeline.predict(X_test)

# If it's regression, evaluate performance with R-squared or MSE
from sklearn.metrics import mean_squared_error, r2_score


#checking for accuracy
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 0.8578413423914023
R-squared (R2): 0.5846204249375766


In [ ]:
train_score = pipeline.score(X_train, y_train)
print(f"Training Accuracy/Score: {train_score}")

Training Accuracy/Score: 0.9112123696208596


In [ ]:
#This data will be recieved from frontend when users input their information
new_entry = pd.DataFrame([{
    "Subject": "Engineering",  # Change subject as needed
    "Skill_Level": 8,               # Adjust based on user skill
    "Task_Difficulty": 7,           # Difficulty of the task
    "Score": 100                     # Desired Grade (in hours)
    }])

In [ ]:
predicted_hours = pipeline.predict(new_entry)
print(f"Predicted Study Hours: {predicted_hours[0]}")

Predicted Study Hours: 3.433928373015872


In [ ]:
X_Train_transformed = pipeline.named_steps["preprocessor"].transform(X_train)

# Get feature names from the transformer
feature_names = pipeline.named_steps["preprocessor"].get_feature_names_out()

# Convert to a DataFrame
X_train_processed = pd.DataFrame(X_Train_transformed, columns=feature_names)

X_train_processed

,num_scaler__Skill_Level,num_scaler__Task_Difficulty,num_scaler__Score,onehot__Subject_Biology,onehot__Subject_Chemistry,onehot__Subject_Computer Science,onehot__Subject_Criminal Justice,onehot__Subject_Cybersecurity,onehot__Subject_Engineering,onehot__Subject_English,onehot__Subject_History,onehot__Subject_Math
0,0.872218,-0.764525,0.547274,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.220291,1.130342,1.378251,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.872218,1.509316,0.859812,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.568363,1.509316,1.378251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.520073,0.372395,-0.386460,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
119995,0.524145,0.372395,0.508110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
119996,0.872218,-1.143499,0.724870,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
119997,1.220291,-0.006578,1.378251,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
119998,0.176073,-1.901446,0.010610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Apply preprocessing only
importances = pd.Series(model.feature_importances_, index=X.columns)
importances.sort_values(ascending=False).plot(kind="bar", title="Feature Importance")
# If you want to check each feature's importance, Find the Blocks that contain the Random Regressor model.
#This is needed because the code above does not work with the pipeline

AttributeError: 'Pipeline' object has no attribute 'feature_importances_'